In [3]:
!pip install tf-keras-vis

In [4]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import tensorflow as tf
from tf_keras_vis.utils import num_of_gpus

_, gpus = num_of_gpus()
print('Tensorflow recognized {} GPUs'.format(gpus))

Tensorflow recognized 1 GPUs


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from tensorflow.keras.applications.vgg16 import VGG16 as Model

# Load model
model = Model(weights='imagenet', include_top=True)
model.summary()

553467096/553467096 [==============================] - 6s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [6]:
from tf_keras_vis.activation_maximization import ActivationMaximization
from tf_keras_vis.activation_maximization.input_modifiers import Jitter, Rotate2D, Scale
from tf_keras_vis.activation_maximization.regularizers import Norm, TotalVariation2D
from tf_keras_vis.activation_maximization.callbacks import Progress, PrintLogger
from tf_keras_vis.utils.model_modifiers import ExtractIntermediateLayer, ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore

def visualize_layer(model, layer_name, scores):
  def model_modifier_function(current_model):
    target_layer = current_model.get_layer(name=layer_name)
    target_layer.activation = tf.keras.activations.linear
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    return new_model

  activation_maximization = ActivationMaximization(
      model,
      model_modifier=model_modifier_function,
      clone=True,
  )

  seed_input = tf.random.uniform((len(scores.indices), 224, 224, 3), 0, 255)

  # Generate maximized activation
  activations = activation_maximization(
      scores,
      seed_input=seed_input,
      callbacks=[Progress()],
  )

  return activations

In [17]:
from tf_keras_vis.activation_maximization.callbacks import Callback
from tf_keras_vis.activation_maximization import ActivationMaximization
from tf_keras_vis.activation_maximization.input_modifiers import Jitter, Rotate2D, Scale
from tf_keras_vis.activation_maximization.regularizers import Norm, TotalVariation2D
from tf_keras_vis.activation_maximization.callbacks import Progress, PrintLogger
from tf_keras_vis.utils.model_modifiers import ExtractIntermediateLayer, ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore

class debug_callback(Callback):
  def on_begin(self, **kwargs) -> None:
    pass

  def on_end(self, **kwargs):
    pass

  def __call__(self, step, clipped_value, grads, score_values, outputs, regularizations, overall_score):
    # print(f'clipped value :{clipped_value.shape}')
    print(f'scores value  :{score_values}')
    # print(f'outputs       :{outputs.shape}')
    print(f'overall_score : {overall_score}')
    # raise "oops"

def generate_gif(model, score: int, category_name='', layer_name = -1):
  from tf_keras_vis.activation_maximization.callbacks import GifGenerator2D

  scores = CategoricalScore([score, score+1])
  seed_input = tf.random.uniform((2, 224, 224, 3), 0, 255)

  # def animate_vis_layer(model, layer_name, seed_input, scores, image_titles):
  activation_maximization = ActivationMaximization(
      model,
      model_modifier=[ExtractIntermediateLayer(index_or_name=layer_name), ReplaceToLinear()],
      clone=True,
  )

  # Generate maximized activation
  activations = activation_maximization(
      scores,
      seed_input=seed_input,
      callbacks=[
                #  Progress(),
                 debug_callback(),
                 GifGenerator2D(f'/content/drive/MyDrive/Vis/{category_name}.gif')
      ],
  )

# image_titles = ['great_white_shark', 'bald_eagle', 'assault_rifle']

generate_gif(model, 292, 'Tiger')
generate_gif(model, 309, 'Bee')
generate_gif(model, 817, 'Car')

scores value  :[<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.1637104, -0.4162066], dtype=float32)>]
overall_score : [<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2.1637104, 0.4162066], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([91.4352 , 91.16737], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([38.254852, 38.25843 ], dtype=float32)>]
scores value  :[<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-1.7689179 ,  0.18179037], dtype=float32)>]
overall_score : [<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 1.7689179 , -0.18179037], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([60.01551, 60.02575], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([38.252163, 38.253742], dtype=float32)>]
scores value  :[<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-1.7989668,  1.5440063], dtype=float32)>]
overall_score : [<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 1.7989668, -